In [1]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt

from car import Car
from tools_cv import *
from tools_kinematics import *

In [28]:
# Constants
A = sp.symbols('A}')
B = sp.symbols('B')
C = sp.symbols('C')
l = sp.symbols('l')
r = sp.symbols('r')

# L_FL = sp.symbols(r'L_{FL}')
# L_FR = sp.symbols(r'L_{FR}')
# L_RL = sp.symbols(r'L_{RL}')
# L_RR = sp.symbols(r'L_{RR}')

# alpha_FL = sp.symbols(r'\alpha_{FL}')
# alpha_FR = sp.symbols(r'\alpha_{FR}')
# alpha_RL = sp.symbols(r'\alpha_{RL}')
# alpha_RR = sp.symbols(r'\alpha_{RR}')

beta_FL = sp.symbols(r'\beta_{FL}')
beta_FR = sp.symbols(r'\beta_{FR}')
# beta_RL = sp.symbols(r'\beta_{RL}')
# beta_RR = sp.symbols(r'\beta_{RR}')

vel_FL = sp.symbols(r"\dot{\phi_{FL}}")
vel_FR = sp.symbols(r"\dot{\phi_{FR}}")
vel_RL = sp.symbols(r"\dot{\phi_{RL}}")
vel_RR = sp.symbols(r"\dot{\phi_{R}}")

theta = sp.symbols(r'\theta')

alias = {}

# display cos(theta) as c\theta, sin(theta) as s\theta
alias.update({sp.sin(theta): sp.symbols(r"s\theta"),
              sp.cos(theta): sp.symbols(r"c\theta"),
              sp.sin(beta_FL): sp.symbols(r"s\beta_{FL}"),
              sp.cos(beta_FL): sp.symbols(r"c\beta_{FL}"),
              sp.sin(beta_FR): sp.symbols(r"s\beta_{FR}"),
              sp.cos(beta_FL): sp.symbols(r"c\beta_{FR}")})

In [29]:
L_FL = L_FR = B / 2
L_RL = L_RR = sp.sqrt(B**2 / 4 + l**2)

alpha_FL = sp.pi/2
alpha_FR = -sp.pi/2
alpha_RL = sp.pi/2 + sp.atan(2*l/B)
alpha_RR = -sp.pi/2 - sp.atan(2*l/B)

beta_RL = -sp.atan(2*l/B)
beta_RR = sp.atan(2*l/B) + sp.pi

In [30]:
# rolling condition
M_rolling_FL = get_mat_rolling(alpha_FL, beta_FL, L_FL)
M_rolling_FR = get_mat_rolling(alpha_FR, beta_FR, L_FR)
M_rolling_RL = get_mat_rolling(alpha_RL, beta_RL, L_RL)
M_rolling_RR = get_mat_rolling(alpha_RR, beta_RR, L_RR)

M_rolling = sp.Matrix([M_rolling_FL, M_rolling_FR,
                       M_rolling_RL, M_rolling_RR])
symprint('M', '', 'rolling')
matprint(M_rolling, alias)

# so sliding condition
M_no_sliding_FL = get_mat_no_sliding(alpha_FL, beta_FL, L_FL)
M_no_sliding_FR = get_mat_no_sliding(alpha_FR, beta_FR, L_FR)
M_no_sliding_RL = get_mat_no_sliding(alpha_RL, beta_RL, L_RL)
M_no_sliding_RR = get_mat_no_sliding(alpha_RR, beta_RR, L_RR)

M_no_sliding = sp.Matrix([M_no_sliding_FL, M_no_sliding_FR,
                          M_no_sliding_RL, M_no_sliding_RR])
symprint('M', '', 'no-sliding')
matprint(M_no_sliding, alias)

M_rolling

Matrix([
[     c\beta_{FR},  s\beta_{FL},                               -B*c\beta_{FR}/2],
[-cos(\beta_{FR}), -s\beta_{FR},                           -B*cos(\beta_{FR})/2],
[               1,            0, -sqrt(B**2 + 4*l**2)/(2*sqrt(1 + 4*l**2/B**2))],
[               1,            0,  sqrt(B**2 + 4*l**2)/(2*sqrt(1 + 4*l**2/B**2))]])

M_no-sliding

Matrix([
[-s\beta_{FL},      c\beta_{FR},                                  B*s\beta_{FL}/2],
[ s\beta_{FR}, -cos(\beta_{FR}),                                  B*s\beta_{FR}/2],
[           0,                1, -l*sqrt(B**2 + 4*l**2)/(B*sqrt(1 + 4*l**2/B**2))],
[           0,                1, -l*sqrt(B**2 + 4*l**2)/(B*sqrt(1 + 4*l**2/B**2))]])

In [31]:
RI0 = rotation(theta, 'z')
R0I = RI0.T
symprint('R', '0', 'I')
matprint(R0I, alias)

^0R_I

Matrix([
[ c\theta, s\theta, 0],
[-s\theta, c\theta, 0],
[       0,       0, 1]])

In [15]:
M_angle = sp.Matrix([M_rolling_RL, M_rolling_RR, M_no_sliding_RL])
symprint('M', '', 'angle')
matprint(M_angle)

M_r = sp.eye(2)*r
M_r = M_r.row_insert(len(M_r), sp.zeros(1, 2))
symprint('M', '', 'r')
matprint(M_r)

M_vel = sp.Matrix([vel_FL, vel_FR, vel_RL, vel_RR])

M_angle

Matrix([
[1, 0,   -sqrt(B**2 + 4*l**2)/(2*sqrt(1 + 4*l**2/B**2))],
[1, 0,    sqrt(B**2 + 4*l**2)/(2*sqrt(1 + 4*l**2/B**2))],
[0, 1, -l*sqrt(B**2 + 4*l**2)/(B*sqrt(1 + 4*l**2/B**2))]])

M_r

Matrix([
[r, 0],
[0, r],
[0, 0]])

## Forward Kinematics

In [53]:
M_temp = M_angle.inv().multiply(M_r)
matprint(sp.simplify(M_temp))

M_temp = M_temp.multiply(sp.Matrix([vel_RL, vel_RR]))
matprint(sp.simplify(M_temp))

M_fwd = RI0.multiply(M_temp)
M_fwd = sp.simplify(M_fwd)
symprint('M', '', 'forward')
matprint(M_fwd, alias)

Matrix([
[                                         r/2,                                         r/2],
[                                      -l*r/B,                                       l*r/B],
[-r*sqrt(1 + 4*l**2/B**2)/sqrt(B**2 + 4*l**2), r*sqrt(1 + 4*l**2/B**2)/sqrt(B**2 + 4*l**2)]])

Matrix([
[                                              r*(\dot{\phi_{RL}} + \dot{\phi_{R}})/2],
[                                           l*r*(-\dot{\phi_{RL}} + \dot{\phi_{R}})/B],
[r*sqrt((B**2 + 4*l**2)/B**2)*(-\dot{\phi_{RL}} + \dot{\phi_{R}})/sqrt(B**2 + 4*l**2)]])

M_forward

Matrix([
[r*(B*c\theta*(\dot{\phi_{RL}} + \dot{\phi_{R}}) + 2*l*s\theta*(\dot{\phi_{RL}} - \dot{\phi_{R}}))/(2*B)],
[r*(B*s\theta*(\dot{\phi_{RL}} + \dot{\phi_{R}}) - 2*c\theta*l*(\dot{\phi_{RL}} - \dot{\phi_{R}}))/(2*B)],
[                   r*sqrt((B**2 + 4*l**2)/B**2)*(-\dot{\phi_{RL}} + \dot{\phi_{R}})/sqrt(B**2 + 4*l**2)]])

## Inverse Kinematics

In [51]:
M_temp_left = M_r.T.multiply(M_r)

M_temp_right = M_r.T.multiply(M_angle)
matprint(sp.simplify(M_temp_right))

M_temp = M_temp_left.inv().multiply(M_temp_right)
matprint(sp.simplify(M_temp))

M_inverse = M_temp.multiply(R0I)
M_inverse = sp.simplify(M_inverse)
symprint('M', '', 'inverse')
matprint(M_inverse, alias)

Matrix([
[r, 0, -r*sqrt(B**2 + 4*l**2)/(2*sqrt(1 + 4*l**2/B**2))],
[r, 0,  r*sqrt(B**2 + 4*l**2)/(2*sqrt(1 + 4*l**2/B**2))]])

Matrix([
[1/r, 0, -sqrt(B**2 + 4*l**2)/(2*r*sqrt(1 + 4*l**2/B**2))],
[1/r, 0,  sqrt(B**2 + 4*l**2)/(2*r*sqrt(1 + 4*l**2/B**2))]])

M_inverse

Matrix([
[c\theta/r, s\theta/r, -sqrt(B**2 + 4*l**2)/(2*r*sqrt(1 + 4*l**2/B**2))],
[c\theta/r, s\theta/r,  sqrt(B**2 + 4*l**2)/(2*r*sqrt(1 + 4*l**2/B**2))]])